In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [3]:
ROOT = "/Users/kkalyan/github/airQo/"
train = pd.read_csv(ROOT+ "data/train-proc.csv")
test = pd.read_csv(ROOT+ "data/test-proc.csv")

In [4]:
test.drop("target",axis  = 1,inplace  = True)

In [5]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [6]:
lab.fit(train["location"])

LabelEncoder()

In [7]:
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [8]:
X=train.drop(["ID",'target'],axis =1)
y = train.target

https://github.com/catboost/benchmarks/blob/master/quality_benchmarks/catboost_experiment.py
https://towardsdatascience.com/deep-dive-into-catboost-functionalities-for-model-interpretation-7cdef669aeed

In [15]:
default_params = {
    "depth": 6,
    "learning_rate": 0.03,
    "iterations": 1000,
    "l2_leaf_reg": 3,
    "eval_metric": "RMSE",
    "n_estimators": 
    use_best_model=True
    
}

In [10]:
params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'ctr_border_count':[50,5,10,20,100,200],
          'thread_count':4}

params = {'depth':[2, 3, 4],
          'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg':np.logspace(-20, -19, 3)
          
    }

parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [30, 50, 100]
                 }
cat_params = {
                'n_estimators':5000,
                'learning_rate': 0.07,
                'eval_metric':'AUC',
                'loss_function':'Logloss',
                'random_seed':SEED,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 8,
                #'colsample_bylevel':0.7,
                } 

clf = CatBoostClassifier(
                               loss_function='MultiClass',
    #                            eval_metric="AUC",
                               task_type="CPU",
                               learning_rate=0.01,
                               iterations=2000,
                               od_type="Iter",
#                                depth=8,
                               early_stopping_rounds=500,
    #                            l2_leaf_reg=1,
    #                            border_count=96,
                               random_seed=2019
                              )
CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [ ]:
cb_model = CatBoostRegressor(iterations=500,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [14]:
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(n_estimators=1000,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.1433360	test: 40.1433360	test1: 40.7384127	best: 40.7384127 (0)	total: 150ms	remaining: 1m 14s
100:	learn: 23.1948628	test: 23.1948628	test1: 27.1768234	best: 27.1575063 (98)	total: 4.8s	remaining: 19s
200:	learn: 18.8846850	test: 18.8846850	test1: 26.0294536	best: 26.0294536 (200)	total: 9.6s	remaining: 14.3s
300:	learn: 15.9202020	test: 15.9202020	test1: 25.5578797	best: 25.5578797 (300)	total: 14.4s	remaining: 9.52s
400:	learn: 13.6686472	test: 13.6686472	test1: 25.0478203	best: 25.0360925 (399)	total: 20.3s	remaining: 5.01s
499:	learn: 11.8744758	test: 11.8744758	test1: 24.8506478	best: 24.8506478 (499)	total: 26.1s	remaining: 0us

bestTest = 24.85064783
bestIteration = 499

err:  24.85064795784145
0:	learn: 40.3193503	test: 40.3193503	test1: 37.4711715	best: 37.4711715 (0)	total: 93ms	remaining: 46.4s
100:	learn: 23.0612601	test: 23.0612601	test1: 24.3917201	best: 24.3917201 (100)	total: 5.16s	remaining: 20.4s
200:	learn: 18.6416117	test: 18.6416117	test1: 23.8277709	

25.362666078093348

In [19]:
from datetime import datetime
exp_time = str(datetime.now().replace(second=0, microsecond=0)).replace(" ", "_")
exp_name = "catboost"
csv_file_name = "{}_{}".format(exp_time, exp_name)

In [20]:
test_id  = test['ID']
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [21]:
SUB = "/Users/kkalyan/github/airQo/submissions/"

In [23]:
sub.to_csv(SUB+csv_file_name+'.csv', index=False)